# Parsing GenBank files

Without specification, the default GenBank parsing function will be used. This function relies on the `locus_tag` field present on every child of a gene feature.

In [ ]:
from inscripta.biocantor.io.genbank.parser import parse_genbank

In [ ]:
gbk = "tests/data/INSC1006_chrI.gbff"

In [ ]:
with open(gbk, "r") as fh:
    parsed = list(parse_genbank(fh))

After parsing, there will be one `ParsedAnnotationRecord` built for every sequence in the GenBank file. This container class holds the original BioPython `SeqRecord` object, as well as one `AnnotationCollectionModel` for the parsed understanding of the annotations. These model objects are `marshmallow_dataclass` objects, and so can be dumped to and loaded directly from JSON.

In [ ]:
annotation_collection_model = parsed[0].annotation
annotation_collection_model.Schema().dump(annotation_collection_model)

OrderedDict([('feature_collections', None),
             ('genes',
              [OrderedDict([('transcripts',
                             [OrderedDict([('exon_starts', [16174]),
                                           ('exon_ends', [18079]),
                                           ('strand', 'MINUS'),
                                           ('cds_starts', None),
                                           ('cds_ends', None),
                                           ('cds_frames', None),
                                           ('qualifiers',
                                            {'ncRNA_class': ['other'],
                                             'locus_tag': ['GI526_G0000001'],
                                             'product': ['CAT novel prediction: IsoSeq'],
                                             'note': ['CAT transcript id: T0000001; CAT alignment id: IsoSeq-PB.2586.1; CAT novel prediction: IsoSeq']}),
                                           ('

## Converting models to BioCantor data structures
After loading an `AnnotationCollectionModel`, this object can be directly converted in to an `AnnotationCollection` with sequence information.

`AnnotationCollection` objects are the core data structure, and contain a set of genes and features as children.

In [ ]:
annotation_collection = parsed[0].to_annotation_collection()

In [ ]:
# this example dataset has 4 genes and 0 features
for child in annotation_collection:
    print(child)

GeneInterval(TranscriptInterval((16174-18079:-), cds=[None], qualifiers={'ncRNA_class': ['other'], 'locus_tag': ['GI526_G0000001'], 'product': ['CAT novel prediction: IsoSeq'], 'note': ['CAT transcript id: T0000001; CAT alignment id: IsoSeq-PB.2586.1; CAT novel prediction: IsoSeq']}))
GeneInterval(TranscriptInterval((37461-39103:+), cds=[CDS((37637-39011:+), (CDSFrame.ZERO)], qualifiers={'gene': ['GDH3'], 'locus_tag': ['GI526_G0000002'], 'product': ['GDH3 isoform 1'], 'note': ['CAT transcript id: T0000002; CAT alignment id: NM_001178204.1-0; CAT source transcript id: NM_001178204.1; CAT source transcript biotype: protein_coding']}))
GeneInterval(TranscriptInterval((39518-40772:+), cds=[CDS((39518-40772:+), (CDSFrame.ZERO)], qualifiers={'gene': ['BDH2'], 'locus_tag': ['GI526_G0000003'], 'codon_start': ['1'], 'product': ['BDH2 isoform 1'], 'protein_id': ['KAF1903246.1'], 'translation': ['MRALAYFGKGNIRFTNHLKEPHIVAPDELVIDIAWCGICGTDLHEYTDGPIFFPEDGHTHEISHNPLPQAMGHEMAGTVLEVGPSVKNLKVGDKVVVEPTG

In [ ]:
gene1 = annotation_collection.genes[0]
tx1 = gene1.transcripts[0]

In [ ]:
# convert mRNA coordinates to genomic coordinates
tx1.transcript_pos_to_sequence(0)

18078

In [ ]:
# NoncodingTranscriptError is raised when trying to convert CDS coordinates on a non-coding transcript
tx1.cds_pos_to_sequence(0)

NoncodingTranscriptError: No CDS positions on non-coding transcript

## Primary transcripts

It is often useful to have an understanding of what isoform of a gene is the 'most important'. An input dataset can provide this information based on the parser implementation used. If this information is not provided, then this value is inferred by the simple heuristic of:

1. Longest CDS isoform.
2. Longest isoform (if no coding isoforms).

In [ ]:
gene1.get_primary_transcript() == tx1

True

## Incorporating sequence information

By default, the instantiation call `ParsedAnnotationRecord.to_annotation_collection` incorporated the sequence information on the objects. This allows for extraction of various types of sequences, including amino acid and spliced transcripts.

In [ ]:
for gene in annotation_collection:
    for tx in gene.transcripts:
        if tx.is_coding:
            print(f"{tx.transcript_name}: mRNA: {tx.get_spliced_sequence()[:10]}... protein: {tx.get_protein_sequence()[:10]}...")
        else:
            print(f"{tx.transcript_name}: mRNA: {tx.get_spliced_sequence()[:10]}...")

None: mRNA: GCGGCGCTCT...
GDH3: mRNA: AAACAGTTAA... protein: MTSEPEFQQA...
BDH2: mRNA: ATGAGAGCCT... protein: MRALAYFGKG...
BDH1: mRNA: GGGGCAGATA...


## Querying the collection

`AnnotationCollections` have the ability to be subsetted. These range queries can be performed in two modes, controlled by the flag `completely_within`. When `completely_within = True`, the positions in the query are exact bounds. When `completely_within = False`, any constituent object that overlaps the range query will be retained.

`start` and `end` are not required to be set, and are inferred to be `0` and `len(sequence)` respectively if not used.

In [ ]:
# remove GI526_G0000001 by moving the start position to within its bounds, when strict boundaries are required
subset1 = annotation_collection.query_by_position(start=16175, completely_within=True)
print([x.locus_tag for x in subset1])

['GI526_G0000002', 'GI526_G0000003', 'GI526_G0000004']


In [ ]:
# the information on the current range of the object is retained
print(subset1.start, subset1.end, subset1.completely_within)

16175 50040 True


In [ ]:
# select BDH1 and BDH2
subset2 = annotation_collection.query_by_position(start=40000, end=42000, completely_within=False)
print([x.gene_symbol for x in subset2])

['BDH2', 'BDH1']
